# Houston 311 Service Requests Data Mining Project

## Group members
Stephen Huang

Levi Villarreal

Joshua Wong

Andrew Young


### Code Imports

In [33]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
import time
import datetime

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import accuracy_score

### Import data

In [34]:

# Data is from http://www.houstontx.gov/311/
names = ["CASE NUMBER", "SR LOCATION", "COUNTY", "DISTRICT", "NEIGHBORHOOD", "TAX ID", 
         "TRASH QUAD", "RECYCLE QUAD", "TRASH DAY", "HEAVY TRASH DAY", "RECYCLE DAY", 
         "KEY MAP", "MANAGEMENT DISTRICT", "DEPARTMENT", "DIVISION", "SR TYPE", "QUEUE", 
         "SLA", "STATUS", "SR CREATE DATE", "DUE DATE", "DATE CLOSED", "OVERDUE", "TITLE", 
         "x", "y", "LATITUDE", "LONGITUDE", "CHANNEL TYPE"]

data = pd.read_csv('311-Public-Data-Extract-2018-clean.txt', sep="|", header=None, names=names)
print(data.shape)
data.head()

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (17,22,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(399953, 29)


,CASE NUMBER,SR LOCATION,COUNTY,DISTRICT,NEIGHBORHOOD,TAX ID,TRASH QUAD,RECYCLE QUAD,TRASH DAY,HEAVY TRASH DAY,...,SR CREATE DATE,DUE DATE,DATE CLOSED,OVERDUE,TITLE,x,y,LATITUDE,LONGITUDE,CHANNEL TYPE
0,CASE NUMBER,SR LOCATION,COUNTY,DISTRICT,NEIGHBORHOOD,TAX ID,TRASH QUAD,RECYCLE QUAD,TRASH DAY,HEAVY TRASH DAY,...,SR CREATE DATE,DUE DATE,DATE CLOSED,OVERDUE,Title,x,y,LATITUDE,LONGITUDE,Channel Type
1,101002866096,Intersection 13300 HEMPSTEAD RD&9100 PINEMONT ...,Harris County,A,FAIRBANKS / NORTHWEST CROSSING,NaN,NaN,NaN,NaN,NaN,...,2018-01-01 00:38:05,2018-01-02 00:38:05,2018-01-01 01:31:49,-0.96,Traffic Signal Maintenance-101002866096,3073146.72528000000,13869806.76417000000,29.84103211,-95.51509863,Voice In
2,12202635-101002866097,"2803 W BELLFORT, HOUSTON TX 77054",NaN,K,SOUTH MAIN,NaN,NaN,NaN,NaN,NaN,...,2018-01-01 00:41:49,2018-01-11 00:41:49,2018-01-01 05:45:02,-9.79,Water Leak-101002866097,3104254.38025000000,13809501.20958000000,29.67268442,-95.42283905,Voice In
3,12202636-101002866098,"2115 HUTTON, HOUSTON TX 77026",HARRIS,B,GREATER FIFTH WARD,0170560000001,NE,NW,FRIDAY,1st Thursday,...,2018-01-01 00:45:07,2018-01-03 00:45:07,2018-01-01 09:55:02,-1.62,Water Service-101002866098,3131779.00434000000,13849821.40912000000,29.7811689,-95.33222248,Voice In
4,12202637-101002866099,"14806 STEEPLE CHASE, HOUSTON TX 77489",FORT BEND,K,FORT BEND HOUSTON,9550030090110907,SW,SW,THURSDAY,3rd Thursday,...,2018-01-01 00:51:20,2018-01-06 00:51:20,2018-01-01 16:10:02,-4.36,Water Main Valve-101002866099,3079284.91871000000,13786008.44784000000,29.61018299,-95.50364663,Voice In


## Dimensionality Reduction
Right off the bat, we can see that there are many features that do not matter at all to our project.
We can go ahead and get rid of these to help prevent against the curse of dimensionality.

In [35]:
data = data.drop(columns=['SR LOCATION', 'CASE NUMBER', 'TRASH QUAD', 'RECYCLE QUAD', 'TRASH DAY', 'HEAVY TRASH DAY', 'RECYCLE DAY', 'TAX ID', 'KEY MAP', 'DUE DATE', 'DATE CLOSED', 'TITLE', 'x', 'y', 'LONGITUDE', 'LATITUDE'])
print(data.shape)

data.head()

(399953, 13)


,COUNTY,DISTRICT,NEIGHBORHOOD,MANAGEMENT DISTRICT,DEPARTMENT,DIVISION,SR TYPE,QUEUE,SLA,STATUS,SR CREATE DATE,OVERDUE,CHANNEL TYPE
0,COUNTY,DISTRICT,NEIGHBORHOOD,MANAGEMENT DISTRICT,DEPARTMENT,DIVISION,SR TYPE,QUEUE,SLA,STATUS,SR CREATE DATE,OVERDUE,Channel Type
1,Harris County,A,FAIRBANKS / NORTHWEST CROSSING,Spring Branch MD,PWE Public Works Engineering,Traffic Operations,Traffic Signal Maintenance,TT_SignalMainDispatch,1,Closed,2018-01-01 00:38:05,-0.96,Voice In
2,NaN,K,SOUTH MAIN,NaN,PWE Public Works Engineering,PU Public Utilities,Water Leak,PU_Water,10,Closed,2018-01-01 00:41:49,-9.79,Voice In
3,HARRIS,B,GREATER FIFTH WARD,NaN,PWE Public Works Engineering,PU Public Utilities,Water Service,PU_Water,2,Closed,2018-01-01 00:45:07,-1.62,Voice In
4,FORT BEND,K,FORT BEND HOUSTON,NaN,PWE Public Works Engineering,PU Public Utilities,Water Main Valve,PU_Water,5,Closed,2018-01-01 00:51:20,-4.36,Voice In


### Reasoning behind dropping the attributes we did

Case number - This was dropped because the case number is a unique identifier assigned to every 311 request, and thus could not be used to classify anything.

Trash Quad - There are requests that are not related to trash collection. Those requests might be misclassified because they are in a different trash quad as other requests in the dataset.

Recycle Quad - Same as Trash Quad

Trash Day - Same as Trash Quad

Heavy Trash Day - Same as Trash Quad

Recycle day - Same as Trash Quad
 
SR Location - Same reasoning as case number.
 
Tax ID - This ID is unique to the property that requested it.
 
Key Map - This attribute is used in the cities internal system to keep track of requests and does not have any actual relation to the request
 
Due Date - We are interested in predicting overdueness, and that it its own seperate column.
 
Date Closed - Same reasoning as due date.
 
Title - This is a another unique identifier assigned to every 311 request, and thus could not be used to classify anything.
 
x - We already have latitude, and this attribute an approximation of latitude.
 
y - We already have longitude, and this attribute an approximation of longitude.

latitude - This is too speficific for our needs, and we have other location metrics.
 
longitude - same as latitude

## Data Cleaning

Drop all service requests that are not closed

In [36]:
oldNumRows = data.shape[0]
data = data[data.STATUS == 'Closed']
data = data.drop(["STATUS"], axis=1)

print("Number of rows dropped: ", oldNumRows - data.shape[0])

Number of rows dropped:  198


Normalize all counties and drop rows where county is unknown

In [37]:
print('Counties:')
print(data['COUNTY'].value_counts())
print()

oldNumRows = data.shape[0]
data['COUNTY'] = data['COUNTY'].replace(['HARRIS', 'FORT BEND', 'MONTGOMERY'], ['Harris County', 'Fort Bend County', 'Montgomery County'])
data = data[data.COUNTY != 'Unknown']

print('Counties:')
print(data['COUNTY'].value_counts())
print()
print("Number of rows with COUNTY == 'Unknown' dropped: ", oldNumRows - data.shape[0])

Counties:
HARRIS               322105
Harris County         43803
FORT BEND              6868
Unknown                3201
Fort Bend County        429
MONTGOMERY              180
Montgomery County       145
Galveston County         13
Waller County             1
Name: COUNTY, dtype: int64

Counties:
Harris County        365908
Fort Bend County       7297
Montgomery County       325
Galveston County         13
Waller County             1
Name: COUNTY, dtype: int64

Number of rows with COUNTY == 'Unknown' dropped:  3201


Drop rows where district is unknown

In [38]:
oldNumRows = data.shape[0]
data = data[data.DISTRICT != 'Unknown']

print("Number of rows dropped: ", oldNumRows - data.shape[0])

Number of rows dropped:  552


Normalize all districts

In [39]:
data['MANAGEMENT DISTRICT'] = data['MANAGEMENT DISTRICT'].replace(['HCID #3 TRACT 19 (Upper Kirby)', 'HCID #3 TRACT 47 (Upper Kirby)', 'East End MD', 'Greater Northside', 'Sharpstown'], ['HCID #3 (Upper Kirby)', 'HCID #3 (Upper Kirby)', 'East End', 'Greater Northside MD', 'Sharpstown MD'])
print('Management Districts:')
print(data['MANAGEMENT DISTRICT'].value_counts().sort_index())

Management Districts:
Airline IMPROVEMENT DISTRICT                   23
Airline PID                                     4
Aldin North Expansion Tract2                    6
Aldine North Expansion Tract1                  18
Aldine PID                                    232
Baybrook MD                                   110
East Downtown MD                             1743
East End                                    16766
FB CAD #6                                       6
Five Corners HCID #10B                      11645
Five Corners Improvement Dist(HCID #10B)       59
Greater Greenspoint MD                       1834
Greater Northside MD                        32036
Greater Southeast MD                        12220
HCID #10-A                                  14007
HCID #12                                        6
HCID #16                                        4
HCID #3 (Upper Kirby)                        1742
HCID #4 (Energy Corridor)                     956
HCID #5 (Brays Oaks)        

Drop all rows with unknown channel types

In [40]:
oldNumRows = data.shape[0]
data = data[data['CHANNEL TYPE'] != 'Unknown']

print("Number of rows dropped: ", oldNumRows - data.shape[0])

Number of rows dropped:  20588


Drop all rows with null values

In [41]:
oldNumRows = data.shape[0]
data = data.dropna()

print("Number of rows dropped: ", oldNumRows - data.shape[0])

Number of rows dropped:  223202


## Feature Transformation

Currently, the request date would be draw conclusions from, so we transformed the feature to instead be the month in which the service request was created, to account seasonal differences in service request time.

In [42]:
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
data['SR CREATE DATE'] = data['SR CREATE DATE'].apply(lambda x : datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
data['SR CREATE DATE'] = data['SR CREATE DATE'].apply(lambda x : months[x.month-1])

data = data.rename(columns={'SR CREATE DATE': 'SR MONTH'})
data.head()

,COUNTY,DISTRICT,NEIGHBORHOOD,MANAGEMENT DISTRICT,DEPARTMENT,DIVISION,SR TYPE,QUEUE,SLA,SR MONTH,OVERDUE,CHANNEL TYPE
1,Harris County,A,FAIRBANKS / NORTHWEST CROSSING,Spring Branch MD,PWE Public Works Engineering,Traffic Operations,Traffic Signal Maintenance,TT_SignalMainDispatch,1,January,-0.96,Voice In
5,Harris County,A,FAIRBANKS / NORTHWEST CROSSING,Spring Branch MD,PWE Public Works Engineering,Traffic Operations,Traffic Signal Maintenance,TT_SignalMainDispatch,1,January,-0.99,Voice In
10,Harris County,I,GREATER HOBBY AREA,HCID #9,PWE Public Works Engineering,Street and Drainage,Street Hazard,ROWM_StreetMain,10,January,-8.83,Voice In
16,Harris County,D,SUNNYSIDE,Five Corners HCID #10B,NS Neighborhood Services,Investigations,Nuisance On Property,NS_Dispatch,180,January,134.19,Voice In
17,Harris County,A,SPRING BRANCH NORTH,Spring Branch MD,SWM Solid Waste Management,General Support Services,Dumpster Complaint,SWM_Complaints,21,January,-0.70,Voice In


Currently, the overdue is continuous, which makes it hard to use in classification algorithms. To combat this, we want to bin the values, so that there are only a few possible values.

The values we want bin into are on time tasks (negative values), tasks done within a week past due date (0 < x < 7), tasks done within a month past due date (7 <= x < 30), tasks done over a month past due date (x >= 30).

In [43]:
def bin_overdue (val):
    val = float(val)
    
    if val < 0:
        return "On Time"
    elif val < 7:
        return "Week"
    elif val < 30:
        return "Month"
    else:
        return "More"
    
data["OVERDUE"] = data["OVERDUE"].apply(bin_overdue)
    
print(data['OVERDUE'].value_counts().sort_index())
data.head()

Month       14224
More         9463
On Time    109965
Week        18560
Name: OVERDUE, dtype: int64


,COUNTY,DISTRICT,NEIGHBORHOOD,MANAGEMENT DISTRICT,DEPARTMENT,DIVISION,SR TYPE,QUEUE,SLA,SR MONTH,OVERDUE,CHANNEL TYPE
1,Harris County,A,FAIRBANKS / NORTHWEST CROSSING,Spring Branch MD,PWE Public Works Engineering,Traffic Operations,Traffic Signal Maintenance,TT_SignalMainDispatch,1,January,On Time,Voice In
5,Harris County,A,FAIRBANKS / NORTHWEST CROSSING,Spring Branch MD,PWE Public Works Engineering,Traffic Operations,Traffic Signal Maintenance,TT_SignalMainDispatch,1,January,On Time,Voice In
10,Harris County,I,GREATER HOBBY AREA,HCID #9,PWE Public Works Engineering,Street and Drainage,Street Hazard,ROWM_StreetMain,10,January,On Time,Voice In
16,Harris County,D,SUNNYSIDE,Five Corners HCID #10B,NS Neighborhood Services,Investigations,Nuisance On Property,NS_Dispatch,180,January,More,Voice In
17,Harris County,A,SPRING BRANCH NORTH,Spring Branch MD,SWM Solid Waste Management,General Support Services,Dumpster Complaint,SWM_Complaints,21,January,On Time,Voice In


## Export data to a CSV

In [44]:
data.to_csv('houston-311-sanitized.csv', index=False)

## Read in Sanitized Data

In [45]:
data = pd.read_csv('houston-311-sanitized.csv', sep=",", header='infer')
print(data.shape)
data.head()

(152212, 12)


,COUNTY,DISTRICT,NEIGHBORHOOD,MANAGEMENT DISTRICT,DEPARTMENT,DIVISION,SR TYPE,QUEUE,SLA,SR MONTH,OVERDUE,CHANNEL TYPE
0,Harris County,A,FAIRBANKS / NORTHWEST CROSSING,Spring Branch MD,PWE Public Works Engineering,Traffic Operations,Traffic Signal Maintenance,TT_SignalMainDispatch,1.0,January,On Time,Voice In
1,Harris County,A,FAIRBANKS / NORTHWEST CROSSING,Spring Branch MD,PWE Public Works Engineering,Traffic Operations,Traffic Signal Maintenance,TT_SignalMainDispatch,1.0,January,On Time,Voice In
2,Harris County,I,GREATER HOBBY AREA,HCID #9,PWE Public Works Engineering,Street and Drainage,Street Hazard,ROWM_StreetMain,10.0,January,On Time,Voice In
3,Harris County,D,SUNNYSIDE,Five Corners HCID #10B,NS Neighborhood Services,Investigations,Nuisance On Property,NS_Dispatch,180.0,January,More,Voice In
4,Harris County,A,SPRING BRANCH NORTH,Spring Branch MD,SWM Solid Waste Management,General Support Services,Dumpster Complaint,SWM_Complaints,21.0,January,On Time,Voice In


In [46]:
num_data = data.copy()
cat_columns = ["COUNTY", "DISTRICT", "NEIGHBORHOOD", "MANAGEMENT DISTRICT", "DEPARTMENT", "DIVISION",
              "SR TYPE", "QUEUE", "SR MONTH", "CHANNEL TYPE"]

for c in cat_columns:
    prev_column = num_data[c]
    encoder = preprocessing.LabelEncoder()
    num_data[c] = encoder.fit_transform(prev_column)

print(num_data.head())
train, test = train_test_split(num_data, train_size = 0.05, test_size = 0.05)

   COUNTY  DISTRICT  NEIGHBORHOOD  MANAGEMENT DISTRICT  DEPARTMENT  DIVISION  \
0       1         0            19                   30          14        28   
1       1         0            19                   30          14        28   
2       1         8            28                   16          14        27   
3       1         3            70                    4          11        16   
4       1         0            68                   30          15        13   

   SR TYPE  QUEUE    SLA  SR MONTH  OVERDUE  CHANNEL TYPE  
0      123     81    1.0         4  On Time             4  
1      123     81    1.0         4  On Time             4  
2      116     61   10.0         4  On Time             4  
3       81     35  180.0         4     More             4  
4       26     68   21.0         4  On Time             4  


In [47]:
down_data = train

labels = down_data['OVERDUE']
df_features = down_data.drop(['OVERDUE'], axis=1)
print(labels.value_counts().sort_index())

Month       714
More        470
On Time    5525
Week        901
Name: OVERDUE, dtype: int64


## Decision Trees

In [48]:
features_train, features_test, labels_train, labels_test = train_test_split(df_features, labels, test_size = 0.2)
print('Training set size:',len(labels_train))
print('Test set size:    ',len(labels_test))
clf = tree.DecisionTreeClassifier()
clf = clf.fit(features_train, labels_train)
predict_labels_test = clf.predict(features_test)
print('Accuracy of decision tree classifier:',accuracy_score(labels_test, predict_labels_test))

Training set size: 6088
Test set size:     1522
Accuracy of decision tree classifier: 0.7233902759526938


## Support Vector Machines (SVM)

In [49]:
pipe = Pipeline(steps=[('scaler',StandardScaler()),('reduce_dim',PCA()),('clf',SVC(gamma='scale'))])
param_grid = {
    'reduce_dim__n_components': list(range(5, 11)),
    'clf__kernel':('linear','rbf','poly')
}
accuracies = cross_val_score(GridSearchCV(pipe,param_grid,cv=5,scoring='accuracy').fit(df_features,labels), df_features, labels, cv=10)
print('Official accuracy:',np.mean(accuracies))

Official accuracy: 0.7295659715025329


## Neural Networks(NN)

In [ ]:
from sklearn.neural_network import MLPClassifier
pipe = Pipeline(steps=[('scaler',StandardScaler()),('clf',MLPClassifier())])
param_grid = {
    'clf__hidden_layer_sizes':((30,),(40,),(50,),(60,)),
    'clf__activation':('logistic','tanh','relu')
}
accuracies = cross_val_score(GridSearchCV(pipe,param_grid,cv=5,scoring='accuracy').fit(df_features,labels), df_features, labels, cv=5)
print('Official accuracy:',np.mean(accuracies))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, Converge

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, Converge

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, Converge

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, Converge

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, Converge

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, Converge

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, Converge

## k-Nearest Neighbor (KNN)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
pipe = Pipeline(steps=[('scaler',StandardScaler()),('reduce_dim',PCA()),('clf',KNeighborsClassifier(n_neighbors=7))])
accuracies = cross_val_score(pipe,df_features,labels,cv=5)
print('Average accuracy:',np.mean(accuracies))

## Naive Bayes
Naive Bayes is not a good option for this dataset due to correlation between attributes. For example, requests from the same neighborhood would be from the same county.

In [ ]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
accuracies = cross_val_score(clf,df_features,labels,cv=10)
print('Average accuracy:',np.mean(accuracies))

## AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=150)
accuracies = cross_val_score(clf, df_features, labels, cv=5)
print('Average accuracy:',np.mean(accuracies))

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
param_grid = {
    'max_depth':list(range(35,55)),
    'min_samples_leaf':(8,10,12),
    'max_features':('sqrt','log2')
}
accuracies = cross_val_score(GridSearchCV(clf,param_grid,cv=5,scoring='accuracy').fit(df_features,labels), df_features, labels, cv=5)
print('Official accuracy:',np.mean(accuracies))

## Final Model